In [1]:
import os
import shutil
import time

def read_all_file(path):
    output = os.listdir(path)
    file_list = []

    for i in output:
        if os.path.isdir(path+"/"+i):
            file_list.extend(read_all_file(path+"/"+i))
        elif os.path.isfile(path+"/"+i):
            file_list.append(path+"/"+i)

    return file_list

def copy_all_file(file_list, new_path):
    for src_path in file_list:
        file = src_path.split("/")[-1]
        shutil.copyfile(src_path, new_path+"/"+file)
        print("파일 {} 작업 완료".format(file)) # 작업한 파일명 출력
        
        


src_path = "age_val_result" # 기존 폴더 경로
new_path = "age_val_merge" # 옮길 폴더 경로

file_list = read_all_file(src_path)
copy_all_file(file_list, new_path)


파일 0939_1974_44_00000007_D.json 작업 완료
파일 1000_1982_38_00000025_D.json 작업 완료
파일 0939_1974_07_00000047_F.json 작업 완료
파일 0935_1954_40_00000041_F.json 작업 완료
파일 0956_1986_05_00000047_F.json 작업 완료
파일 0967_1977_42_00000010_D.json 작업 완료
파일 0980_1971_52_00000022_D.json 작업 완료
파일 0973_1982_12_00000031_F.json 작업 완료
파일 1000_1982_36_00000020_D.json 작업 완료
파일 1000_1982_27_00000008_D.json 작업 완료
파일 0973_1982_33_00000016_D.json 작업 완료
파일 0973_1982_03_00000044_F.json 작업 완료
파일 0935_1954_63_00000015_D.json 작업 완료
파일 0935_1954_66_00000049_F.json 작업 완료
파일 0956_1986_26_00000010_D.json 작업 완료
파일 0952_1980_10_00000033_F.json 작업 완료
파일 0952_1980_28_00000010_D.json 작업 완료
파일 0940_1979_24_00000046_F.json 작업 완료
파일 0980_1971_52_00000020_D.json 작업 완료
파일 0952_1980_06_00000049_F.json 작업 완료
파일 0967_1977_03_00000038_F.json 작업 완료
파일 0973_1982_41_00000002_F.json 작업 완료
파일 0973_1982_38_00000023_D.json 작업 완료
파일 0980_1971_47_00000003_D.json 작업 완료
파일 0952_1980_29_00000011_D.json 작업 완료
파일 0973_1982_29_00000011_D.json 작업 완료
파일 0980_1971

In [4]:
import os
import json
import shutil
from PIL import Image

unique_classes = set()

# 이미지 및 레이블이 있는 폴더 경로
data_folder = r'age_val_merge'

# YOLO 형식으로 변환한 데이터를 저장할 폴더 경로
output_folder = r'age_val_yolo'

# 이미지 파일 확장자
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

# 현재 클래스들: baseball park, person, steeltower, towercrane, car, bike, truck, ground, helipad, etc vehicle, outdoor parking, bus, high-rise building
# YOLO 클래스 이름과 COCO 클래스 이름 매합 (필요에 따라 업데이트)
class_mapping = {
    "1": 0,
    "2": 1,
    "3": 2,
    "4": 3,
    "5": 4,
    "6": 5,
    "7": 6,
    "8": 7,
    "9": 8,
    "10": 9,
    "11": 10,
    "12": 11,
    "13": 12,
    "14": 13,
    "15": 14,
    "16": 15,
    "17": 16,
    "18": 17,
    "19": 18,
    "20": 19,
    "21": 20,
    "22": 21,
    "23": 22,
    "24": 23,
    "25": 24,
    "26": 25,
    "27": 26,
    "28": 27,
    "29": 28,
    "30": 29,
    "31": 30,
    "32": 31,
    "33": 32,
    "34": 33,
    "35": 34,
    "36": 35,
    "37": 36,
    "38": 37,
    "39": 38,
    "40": 39,
    "41": 40,
    "42": 41,
    "43": 42,
    "44": 43,
    "45": 44,
    "46": 45,
    "47": 46,
    "48": 47,
    "49": 48,
    "50": 49,
    "51": 50,
    "52": 51,
    "53": 52,
    "54": 53,
    "55": 54,
    "56": 55,
    "57": 56,
    "58": 57,
    "59": 58,
    "60": 59,
    "61": 60,
    "62": 61,
    "63": 62,
    "64": 63,
    "65": 64,
    "66": 65,
    "67": 66,
    "68": 67,
    "69": 68
}

# 레이블을 YOLO 형식으로 변환
def coco_to_yolo_label(coco_label, points, width, height):
    yolo_class = class_mapping.get(coco_label, 'unknown')
    if len(points) == 4:
        x_min, y_min, x_max, y_max = points[0], points[1], points[2], points[3]
        x_max = x_min + points[2]
        y_max = y_min + points[3]
        x_center = (x_min + x_max) / (2 * width)
        y_center = (y_min + y_max) / (2 * height)
        box_width = (x_max - x_min) / width
        box_height = (y_max - y_min) / height
        return f"{yolo_class} {x_center} {y_center} {box_width} {box_height}"

# 이미지 파일과 레이블 파일의 수를 계산
total_image_files = 0
total_label_files = 0
error_files = []

for root, _, files in os.walk(data_folder):
    for file in files:
        if file.lower().endswith(tuple(image_extensions)):
            total_image_files += 1
        elif file.lower().endswith('.json'):
            total_label_files += 1

if total_image_files != total_label_files:
    print(f"에러: 이미지와 레이블 카운트가 일치하지 않습니다. (이미지: {total_image_files}, 레이블: {total_label_files})")
    # sys.exit()

# 결과를 저장할 폴더 생성
os.makedirs(output_folder, exist_ok=True)

# 진행 중인 파일 수 초기화
processed_files = 0

# 이미지와 레이블 폴더에서 데이터 확인
for root, _, files in os.walk(data_folder):
    for file in files:
        if file.lower().endswith(tuple(image_extensions)):
            # 이미지 파일 경로
            image_path = os.path.join(root, file)

            # 해당 이미지에 대한 레이블 파일 경로
            label_name = os.path.splitext(file)[0]
            label_file = label_name + '.json'
            label_file_path = os.path.join(root, label_file)

            if os.path.isfile(label_file_path):
                yolo_labels = []  # 이미지에 대한 모든 레이블 정보를 저장하는 리스트
                with open(label_file_path, 'r', encoding='utf-8') as label_data:
                    coco_data = json.load(label_data)
                    # 이미지의 너비와 높이 가져오기
                    try:
                        width = coco_data["width"]
                        height = coco_data["height"]
                    except:
                        print(f"에러: 이미지 해상도를 찾을 수 없습니다 - {label_name}")
                        error_files.append(label_name)
                        continue

                    # 'annotation'이 리스트인 경우에 대한 처리
                    annotations = coco_data.get('annotation', [])
                    if not isinstance(annotations, list):
                        annotations = [annotations]

                    for annotation in annotations:
                        coco_label = coco_data.get('age_past', []) 
                        box_info = annotation.get('box', {})
                        points = [box_info.get('x', 0), box_info.get('y', 0), box_info.get('w', 0), box_info.get('h', 0)]
                        yolo_label = coco_to_yolo_label(str(coco_label), points, width, height)
                        yolo_labels.append(yolo_label)
                yolo_label_file_path = os.path.join(output_folder, label_name + '.txt')
                with open(yolo_label_file_path, 'w') as yolo_label_file:
                    # 모든 레이블 정보를 한 번에 작성
                    for label in yolo_labels:
                        if label:  # 레이블이 있는 경우에만 파일에 쓰도록 수정
                            yolo_label_file.write(f"{label}\n")

                # 이미지 파일을 같은 위치에 복사
                image_copy_path = os.path.join(output_folder, file)
                shutil.copy(image_path, image_copy_path)

                processed_files += 1  # 파일 처리가 완료되면 processed_files를 증가

                # 진행 상황 출력 (백분율로 표시)
                progress_percentage = (processed_files / total_image_files) * 100
                print(f"COCOTOYOLO [{data_folder}->{output_folder}] 진행 상황: {processed_files}/{total_image_files} 레이블 처리 중 ({progress_percentage:.2f}%), 에러: {len(error_files)}")

# 진행 상황 출력
print("COCO 데이터를 YOLO 데이터로 변환 완료.")

if total_image_files != total_label_files:
    print(f"에러: 이미지와 레이블 카운트가 일치하지 않습니다. (이미지: {total_image_files}, 레이블: {total_label_files})")

# 에러 파일 출력
if error_files:
    print("에러 파일 목록:")
    for error_file in error_files:
        print(error_file)


COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 1/450 레이블 처리 중 (0.22%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 2/450 레이블 처리 중 (0.44%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 3/450 레이블 처리 중 (0.67%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 4/450 레이블 처리 중 (0.89%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 5/450 레이블 처리 중 (1.11%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 6/450 레이블 처리 중 (1.33%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 7/450 레이블 처리 중 (1.56%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 8/450 레이블 처리 중 (1.78%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 9/450 레이블 처리 중 (2.00%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 10/450 레이블 처리 중 (2.22%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 11/450 레이블 처리 중 (2.44%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 12/450 레이블 처리 중 (2.67%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 13/450 레이블 처리

COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 139/450 레이블 처리 중 (30.89%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 140/450 레이블 처리 중 (31.11%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 141/450 레이블 처리 중 (31.33%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 142/450 레이블 처리 중 (31.56%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 143/450 레이블 처리 중 (31.78%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 144/450 레이블 처리 중 (32.00%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 145/450 레이블 처리 중 (32.22%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 146/450 레이블 처리 중 (32.44%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 147/450 레이블 처리 중 (32.67%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 148/450 레이블 처리 중 (32.89%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 149/450 레이블 처리 중 (33.11%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 150/450 레이블 처리 중 (33.33%), 에러: 0
COCOTOYOLO [age_val_merge->a

COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 280/450 레이블 처리 중 (62.22%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 281/450 레이블 처리 중 (62.44%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 282/450 레이블 처리 중 (62.67%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 283/450 레이블 처리 중 (62.89%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 284/450 레이블 처리 중 (63.11%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 285/450 레이블 처리 중 (63.33%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 286/450 레이블 처리 중 (63.56%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 287/450 레이블 처리 중 (63.78%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 288/450 레이블 처리 중 (64.00%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 289/450 레이블 처리 중 (64.22%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 290/450 레이블 처리 중 (64.44%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 291/450 레이블 처리 중 (64.67%), 에러: 0
COCOTOYOLO [age_val_merge->a

COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 406/450 레이블 처리 중 (90.22%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 407/450 레이블 처리 중 (90.44%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 408/450 레이블 처리 중 (90.67%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 409/450 레이블 처리 중 (90.89%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 410/450 레이블 처리 중 (91.11%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 411/450 레이블 처리 중 (91.33%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 412/450 레이블 처리 중 (91.56%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 413/450 레이블 처리 중 (91.78%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 414/450 레이블 처리 중 (92.00%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 415/450 레이블 처리 중 (92.22%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 416/450 레이블 처리 중 (92.44%), 에러: 0
COCOTOYOLO [age_val_merge->age_val_yolo] 진행 상황: 417/450 레이블 처리 중 (92.67%), 에러: 0
COCOTOYOLO [age_val_merge->a

In [15]:
print(points)

[203, 493, 597, 597]


In [14]:
#패션 result
import os
import shutil
import time

def read_all_file(path):
    output = os.listdir(path)
    file_list = []

    for i in output:
        if os.path.isdir(path+"/"+i):
            file_list.extend(read_all_file(path+"/"+i))
        elif os.path.isfile(path+"/"+i):
            file_list.append(path+"/"+i)

    return file_list

def copy_all_file(file_list, new_path):
    for src_path in file_list:
        file = src_path.split("/")[-1]
        shutil.copyfile(src_path, new_path+"/"+file)
        print("파일 {} 작업 완료".format(file)) # 작업한 파일명 출력
        
        


src_path = "fashion_result" # 기존 폴더 경로
new_path = "fashion_result_merge" # 옮길 폴더 경로

file_list = read_all_file(src_path)
copy_all_file(file_list, new_path)

파일 1145102.jpg 작업 완료
파일 831752.jpg 작업 완료
파일 995467.jpg 작업 완료
파일 845037.jpg 작업 완료
파일 398698.jpg 작업 완료
파일 187066.jpg 작업 완료
파일 1243277.jpg 작업 완료
파일 399588.jpg 작업 완료
파일 1273332.jpg 작업 완료
파일 1256301.jpg 작업 완료
파일 499760.jpg 작업 완료
파일 1041332.jpg 작업 완료
파일 837385.jpg 작업 완료
파일 190857.jpg 작업 완료
파일 1217453.jpg 작업 완료
파일 1113580.jpg 작업 완료
파일 1291785.jpg 작업 완료
파일 883405.jpg 작업 완료
파일 1238451.jpg 작업 완료
파일 945278.jpg 작업 완료
파일 191390.jpg 작업 완료
파일 1078694.jpg 작업 완료
파일 841644.jpg 작업 완료
파일 749941.jpg 작업 완료
파일 265191.jpg 작업 완료
파일 1177952.jpg 작업 완료
파일 163135.jpg 작업 완료
파일 590485.jpg 작업 완료
파일 260910.jpg 작업 완료
파일 157228.jpg 작업 완료
파일 960907.jpg 작업 완료
파일 995117.jpg 작업 완료
파일 44272.jpg 작업 완료
파일 696841.jpg 작업 완료
파일 1157062.jpg 작업 완료
파일 1119137.jpg 작업 완료
파일 1186094.jpg 작업 완료
파일 703750.jpg 작업 완료
파일 84692.jpg 작업 완료
파일 114078.jpg 작업 완료
파일 1119167.jpg 작업 완료
파일 906900.jpg 작업 완료
파일 1308926.jpg 작업 완료
파일 684697.jpg 작업 완료
파일 190816.jpg 작업 완료
파일 122454.jpg 작업 완료
파일 784597.jpg 작업 완료
파일 578267.jpg 작업 완료
파일 251682.jpg 작업 완료
파일 101

파일 959127.jpg 작업 완료
파일 818117.jpg 작업 완료
파일 909096.jpg 작업 완료
파일 280032.jpg 작업 완료
파일 190516.jpg 작업 완료
파일 1265296.jpg 작업 완료
파일 726034.jpg 작업 완료
파일 1037909.jpg 작업 완료
파일 798298.jpg 작업 완료
파일 851517.jpg 작업 완료
파일 865415.jpg 작업 완료
파일 44232.jpg 작업 완료
파일 1189904.jpg 작업 완료
파일 1263612.jpg 작업 완료
파일 735170.jpg 작업 완료
파일 1240094.jpg 작업 완료
파일 503101.jpg 작업 완료
파일 1036517.jpg 작업 완료
파일 659421.jpg 작업 완료
파일 1308136.jpg 작업 완료
파일 926707.jpg 작업 완료
파일 1026942.jpg 작업 완료
파일 361309.jpg 작업 완료
파일 1193312.jpg 작업 완료
파일 1334497.jpg 작업 완료
파일 18155.jpg 작업 완료
파일 813382.jpg 작업 완료
파일 904748.jpg 작업 완료
파일 877639.jpg 작업 완료
파일 1240084.jpg 작업 완료
파일 1041022.jpg 작업 완료
파일 258925.jpg 작업 완료
파일 493861.jpg 작업 완료
파일 730849.jpg 작업 완료
파일 851437.jpg 작업 완료
파일 1254693.jpg 작업 완료
파일 192418.jpg 작업 완료
파일 113047.jpg 작업 완료
파일 835199.jpg 작업 완료
파일 1012078.jpg 작업 완료
파일 1229065.jpg 작업 완료
파일 1264223.jpg 작업 완료
파일 1148227.jpg 작업 완료
파일 260900.jpg 작업 완료
파일 739070.jpg 작업 완료
파일 996287.jpg 작업 완료
파일 192951.jpg 작업 완료
파일 443416.jpg 작업 완료
파일 1209256.jpg 작업 완료
파일 1

파일 1207186.jpg 작업 완료
파일 79293.jpg 작업 완료
파일 79398.jpg 작업 완료
파일 1036181.jpg 작업 완료
파일 330202.jpg 작업 완료
파일 864405.jpg 작업 완료
파일 813402.jpg 작업 완료
파일 830076.jpg 작업 완료
파일 288692.jpg 작업 완료
파일 764523.jpg 작업 완료
파일 583649.jpg 작업 완료
파일 1270614.jpg 작업 완료
파일 140071.jpg 작업 완료
파일 104844.jpg 작업 완료
파일 1271674.jpg 작업 완료
파일 43404.jpg 작업 완료
파일 452024.jpg 작업 완료
파일 976428.jpg 작업 완료
파일 191669.jpg 작업 완료
파일 1178122.jpg 작업 완료
파일 494095.jpg 작업 완료
파일 61296.jpg 작업 완료
파일 451168.jpg 작업 완료
파일 1089629.jpg 작업 완료
파일 63597.jpg 작업 완료
파일 342688.jpg 작업 완료
파일 563120.jpg 작업 완료
파일 192159.jpg 작업 완료
파일 1211973.jpg 작업 완료
파일 394370.jpg 작업 완료
파일 709337.jpg 작업 완료
파일 191256.jpg 작업 완료
파일 1263412.jpg 작업 완료
파일 622924.jpg 작업 완료
파일 354855.jpg 작업 완료
파일 477374.jpg 작업 완료
파일 474500.jpg 작업 완료
파일 1256331.jpg 작업 완료
파일 1185984.jpg 작업 완료
파일 570790.jpg 작업 완료
파일 1145112.jpg 작업 완료
파일 1083584.jpg 작업 완료
파일 684887.jpg 작업 완료
파일 995107.jpg 작업 완료
파일 1003796.jpg 작업 완료
파일 675521.jpg 작업 완료
파일 158827.jpg 작업 완료
파일 1239483.jpg 작업 완료
파일 875057.jpg 작업 완료
파일 1250574.

파일 156656.jpg 작업 완료
파일 413073.jpg 작업 완료
파일 1238461.jpg 작업 완료
파일 490408.jpg 작업 완료
파일 566626.jpg 작업 완료
파일 192811.jpg 작업 완료
파일 649385.jpg 작업 완료
파일 738750.jpg 작업 완료
파일 982152.jpg 작업 완료
파일 1211803.jpg 작업 완료
파일 1137617.jpg 작업 완료
파일 831098.jpg 작업 완료
파일 191317.jpg 작업 완료
파일 63607.jpg 작업 완료
파일 875067.jpg 작업 완료
파일 1274098.jpg 작업 완료
파일 742162.jpg 작업 완료
파일 1254663.jpg 작업 완료
파일 1138908.jpg 작업 완료
파일 501502.jpg 작업 완료
파일 1239433.jpg 작업 완료
파일 32762.jpg 작업 완료
파일 169448.jpg 작업 완료
파일 192083.jpg 작업 완료
파일 1237891.jpg 작업 완료
파일 399458.jpg 작업 완료
파일 539394.jpg 작업 완료
파일 172247.jpg 작업 완료
파일 900421.jpg 작업 완료
파일 726094.jpg 작업 완료
파일 698285.jpg 작업 완료
파일 192529.jpg 작업 완료
파일 880688.jpg 작업 완료
파일 383451.jpg 작업 완료
파일 703080.jpg 작업 완료
파일 1044067.jpg 작업 완료
파일 1038784.jpg 작업 완료
파일 1270364.jpg 작업 완료
파일 100251.jpg 작업 완료
파일 1271684.jpg 작업 완료
파일 524285.jpg 작업 완료
파일 618046.jpg 작업 완료
파일 161845.jpg 작업 완료
파일 831873.jpg 작업 완료
파일 57166.jpg 작업 완료
파일 1238808.jpg 작업 완료
파일 1270534.jpg 작업 완료
파일 1322390.jpg 작업 완료
파일 161731.jpg 작업 완료
파일 13218

파일 936388.jpg 작업 완료
파일 61141.jpg 작업 완료
파일 1196709.jpg 작업 완료
파일 927027.jpg 작업 완료
파일 191287.jpg 작업 완료
파일 665220.jpg 작업 완료
파일 133320.jpg 작업 완료
파일 1243477.jpg 작업 완료
파일 1263322.jpg 작업 완료
파일 950622.jpg 작업 완료
파일 984548.jpg 작업 완료
파일 1113560.jpg 작업 완료
파일 1311797.jpg 작업 완료
파일 928879.jpg 작업 완료
파일 692329.jpg 작업 완료
파일 930531.jpg 작업 완료
파일 246488.jpg 작업 완료
파일 1133102.jpg 작업 완료
파일 243674.jpg 작업 완료
파일 1011628.jpg 작업 완료
파일 1258065.jpg 작업 완료
파일 622188.jpg 작업 완료
파일 1321900.jpg 작업 완료
파일 1145582.jpg 작업 완료
파일 517346.jpg 작업 완료
파일 1084846.jpg 작업 완료
파일 1279947.jpg 작업 완료
파일 160952.jpg 작업 완료
파일 698115.jpg 작업 완료
파일 47359.jpg 작업 완료
파일 616459.jpg 작업 완료
파일 229487.jpg 작업 완료
파일 1009838.jpg 작업 완료
파일 353761.jpg 작업 완료
파일 984538.jpg 작업 완료
파일 1145122.jpg 작업 완료
파일 1238441.jpg 작업 완료
파일 1119597.jpg 작업 완료
파일 1142651.jpg 작업 완료
파일 162863.jpg 작업 완료
파일 841674.jpg 작업 완료
파일 184958.jpg 작업 완료
파일 982192.jpg 작업 완료
파일 527992.jpg 작업 완료
파일 353750.jpg 작업 완료
파일 850535.jpg 작업 완료
파일 1119537.jpg 작업 완료
파일 574035.jpg 작업 완료
파일 1113590.jpg 작업 완료
파일 

파일 286065.json 작업 완료
파일 616613.json 작업 완료
파일 1011648.json 작업 완료
파일 622574.json 작업 완료
파일 452802.json 작업 완료
파일 1015526.json 작업 완료
파일 1237671.json 작업 완료
파일 1284500.json 작업 완료
파일 663534.json 작업 완료
파일 192645.json 작업 완료
파일 321485.json 작업 완료
파일 173771.json 작업 완료
파일 1058301.json 작업 완료
파일 376098.json 작업 완료
파일 1185656.json 작업 완료
파일 1270374.json 작업 완료
파일 1267459.json 작업 완료
파일 63597.json 작업 완료
파일 369921.json 작업 완료
파일 666212.json 작업 완료
파일 162851.json 작업 완료
파일 839991.json 작업 완료
파일 115804.json 작업 완료
파일 1157042.json 작업 완료
파일 619280.json 작업 완료
파일 888652.json 작업 완료
파일 1027007.json 작업 완료
파일 211600.json 작업 완료
파일 158838.json 작업 완료
파일 730849.json 작업 완료
파일 845197.json 작업 완료
파일 1216933.json 작업 완료
파일 1308236.json 작업 완료
파일 692339.json 작업 완료
파일 1044587.json 작업 완료
파일 1138309.json 작업 완료
파일 732607.json 작업 완료
파일 818127.json 작업 완료
파일 1307271.json 작업 완료
파일 1239172.json 작업 완료
파일 883395.json 작업 완료
파일 72464.json 작업 완료
파일 158874.json 작업 완료
파일 61778.json 작업 완료
파일 1343123.json 작업 완료
파일 906840.json 작업 완료
파일 1285140.json 작업 완

In [3]:
import os
import json
import shutil
import sys
from PIL import Image

def check_category(data, category_name, annotations):
    if "데이터셋 정보" in data \
            and "데이터셋 상세설명" in data["데이터셋 정보"] \
            and "라벨링" in data["데이터셋 정보"]["데이터셋 상세설명"] \
            and category_name in data["데이터셋 정보"]["데이터셋 상세설명"]["라벨링"] \
            and data["데이터셋 정보"]["데이터셋 상세설명"]["라벨링"][category_name]:
        
        try:
            for index in range(len(data["데이터셋 정보"]["데이터셋 상세설명"]["라벨링"][category_name])):
                category_value = data["데이터셋 정보"]["데이터셋 상세설명"]["라벨링"][category_name][index]["카테고리"]
                x = data["데이터셋 정보"]["데이터셋 상세설명"]["렉트좌표"][category_name][index]["X좌표"]
                y = data["데이터셋 정보"]["데이터셋 상세설명"]["렉트좌표"][category_name][index]["Y좌표"]
                width = data["데이터셋 정보"]["데이터셋 상세설명"]["렉트좌표"][category_name][index]["가로"]
                height = data["데이터셋 정보"]["데이터셋 상세설명"]["렉트좌표"][category_name][index]["세로"]

                annotation = {"label": category_value, "points": [x, y, width, height]}
                annotations.append(annotation)

                # print(data["데이터셋 정보"]["데이터셋 상세설명"]["라벨링"][category_name][index]["카테고리"])
                # print(data["데이터셋 정보"]["데이터셋 상세설명"]["렉트좌표"][category_name][index]["X좌표"])
                # print(data["데이터셋 정보"]["데이터셋 상세설명"]["렉트좌표"][category_name][index]["Y좌표"])
                # print(data["데이터셋 정보"]["데이터셋 상세설명"]["렉트좌표"][category_name][index]["가로"])
                # print(data["데이터셋 정보"]["데이터셋 상세설명"]["렉트좌표"][category_name][index]["세로"])

            # if len(data["데이터셋 정보"]["데이터셋 상세설명"]["라벨링"][category_name]) > 1:
            #     print("aaa")
            
        except (IndexError, KeyError):
            pass
            # print(f"{category_name} 카테고리가 존재하지 않거나 인덱스 오류 발생")
    else:
        pass
        # print(f"{category_name}에 해당하는 경로에 필요한 키가 존재하지 않습니다.")



# 이미지 및 레이블이 있는 폴더 경로
data_folder = r'/root/fashion_result_merge'

# YOLO 형식으로 변환한 데이터를 저장할 폴더 경로
output_folder = r'/root/fashion_converter'

# 이미지 파일 확장자
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

# 현재 클래스들: baseball park, person, steeltower, towercrane, car, bike, truck, ground, helipad, etc vehicle, outdoor parking, bus, high-rise building
# YOLO 클래스 이름과 COCO 클래스 이름 매합 (필요에 따라 업데이트)
class_mapping = {
    "탑": "0",
    "점프수트": "1",
    "티셔츠": "2",
    "청바지": "3",
    "셔츠": "4",
    "브라탑": "5",
    "팬츠": "6",
    "후드티": "7",
    "베스트": "8",
    "드레스": "9",
    "조거팬츠": "10",
    "점퍼": "11",
    "코트": "12",
    "스커트": "13",
    "짚업": "14",
    "가디건": "15",
    "재킷": "16",
    "니트웨어": "17",
    "블라우스": "18",
    "래깅스":"18",
    "패딩":"19"
}
# 탑, 점프수트, 티셔츠, 청바지, 셔츠, 브라탑, 팬츠, 후드티, 베스트, 드레스, 조거팬츠, 점퍼, 코트, 스커트, 짚업, 가디건, 재킷, 니트웨어, 블라우스

# 레이블을 YOLO 형식으로 변환
def coco_to_yolo_label(coco_label, points, width, height):
    yolo_class = class_mapping.get(coco_label, 'unknown')
    if yolo_class == 'unknown':
        print("ERROR: class가 매칭되지 않았습니다.")
        sys.exit()
    if len(points) == 4:
        x_min, y_min, bbox_w, bbox_h = points[0], points[1], points[2], points[3]

        # 중심 좌표 계산
        center_x = x_min + bbox_w / 2
        center_y = y_min + bbox_h / 2

        # 중심 좌표를 0에서 1 사이의 값으로 변환
        x_center = center_x / width
        y_center = center_y / height

        # 너비와 높이를 0에서 1 사이의 값으로 변환
        box_width = bbox_w / width
        box_height = bbox_h / height

        return f"{yolo_class} {x_center} {y_center} {box_width} {box_height}"
        

# 이미지 파일과 레이블 파일의 수를 계산
total_image_files = 0
total_label_files = 0
error_files = []

for root, _, files in os.walk(data_folder):
    for file in files:
        if file.lower().endswith(tuple(image_extensions)):
            total_image_files += 1
        elif file.lower().endswith('.json'):
            total_label_files += 1

if total_image_files != total_label_files:
    print(f"에러: 이미지와 레이블 카운트가 일치하지 않습니다. (이미지: {total_image_files}, 레이블: {total_label_files})")
    # sys.exit()

# 결과를 저장할 폴더 생성
os.makedirs(output_folder, exist_ok=True)

# 진행 중인 파일 수 초기화
processed_files = 0

# 이미지와 레이블 폴더에서 데이터 확인
for root, _, files in os.walk(data_folder):
    for file in files:
        if file.lower().endswith(tuple(image_extensions)):
            # 이미지 파일 경로
            image_path = os.path.join(root, file)

            # 해당 이미지에 대한 레이블 파일 경로
            label_name = os.path.splitext(file)[0]
            label_file = label_name + '.json'
            label_file_path = os.path.join(root, label_file)

            if os.path.isfile(label_file_path):
                yolo_labels = []  # 이미지에 대한 모든 레이블 정보를 저장하는 리스트
                annotations = []

                with open(label_file_path, 'r', encoding='utf-8') as label_data:
                    coco_data = json.load(label_data)

                    check_category(coco_data, "아우터", annotations)
                    check_category(coco_data, "원피스", annotations)
                    check_category(coco_data, "상의", annotations)
                    check_category(coco_data, "하의", annotations)

                    # annotations = coco_data.get('annotations', [])
                    
                    # 이미지의 너비와 높이 가져오기
                    try:
                        im = Image.open(image_path)
                        width, height = im.size
                    except:
                        print(f"에러: 이미지 해상도를 찾을 수 없습니다 - {label_name}")
                        error_files.append(label_name)
                        continue

                    # print(f"Label: {label_name}, Width: {width}, Height: {height}")
                    for annotation in annotations:
                        coco_label = annotation['label']
                        
                        annotation_points = annotation['points']
                        yolo_label = coco_to_yolo_label(coco_label, annotation_points, width, height)
                        
                        yolo_labels.append(yolo_label)
                
                yolo_label_file_path = os.path.join(output_folder, label_name + '.txt')
                with open(yolo_label_file_path, 'w') as yolo_label_file:
                    # 모든 레이블 정보를 한 번에 작성
                    for label in yolo_labels:
                        yolo_label_file.write(f"{label}\n")

                # 이미지 파일을 같은 위치에 복사
                image_copy_path = os.path.join(output_folder, file)
                shutil.copy(image_path, image_copy_path)

                processed_files += 1  # 파일 처리가 완료되면 processed_files를 증가

                # 진행 상황 출력 (백분율로 표시)
                progress_percentage = (processed_files / total_image_files) * 100
                
                print(f"COCOTOYOLO [{data_folder}->{output_folder}] 진행 상황: {processed_files}/{total_label_files} 레이블 처리 중 ({progress_percentage:.2f}%), 에러: {len(error_files)}")


# 진행 상황 출력
print("COCO 데이터를 YOLO 데이터로 변환 완료.")

if total_image_files != total_label_files:
    print(f"에러: 이미지와 레이블 카운트가 일치하지 않습니다. (이미지: {total_image_files}, 레이블: {total_label_files})")

# 에러 파일 출력
if error_files:
    print("에러 파일 목록:")
    for error_file in error_files:
        print(error_file)




COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 1/2772 레이블 처리 중 (0.04%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2/2772 레이블 처리 중 (0.07%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 3/2772 레이블 처리 중 (0.11%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 4/2772 레이블 처리 중 (0.14%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 5/2772 레이블 처리 중 (0.18%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 6/2772 레이블 처리 중 (0.22%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 7/2772 레이블 처리 중 (0.25%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 8/2772 레이블 처리 중 (0.29%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 9/2772 레이블 처리 중 (0.32%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 1

COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 129/2772 레이블 처리 중 (4.65%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 130/2772 레이블 처리 중 (4.69%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 131/2772 레이블 처리 중 (4.73%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 132/2772 레이블 처리 중 (4.76%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 133/2772 레이블 처리 중 (4.80%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 134/2772 레이블 처리 중 (4.83%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 135/2772 레이블 처리 중 (4.87%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 136/2772 레이블 처리 중 (4.91%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 137/2772 레이블 처리 중 (4.94%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_c

COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 259/2772 레이블 처리 중 (9.34%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 260/2772 레이블 처리 중 (9.38%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 261/2772 레이블 처리 중 (9.42%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 262/2772 레이블 처리 중 (9.45%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 263/2772 레이블 처리 중 (9.49%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 264/2772 레이블 처리 중 (9.52%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 265/2772 레이블 처리 중 (9.56%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 266/2772 레이블 처리 중 (9.60%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 267/2772 레이블 처리 중 (9.63%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_c

COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 384/2772 레이블 처리 중 (13.85%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 385/2772 레이블 처리 중 (13.89%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 386/2772 레이블 처리 중 (13.92%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 387/2772 레이블 처리 중 (13.96%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 388/2772 레이블 처리 중 (14.00%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 389/2772 레이블 처리 중 (14.03%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 390/2772 레이블 처리 중 (14.07%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 391/2772 레이블 처리 중 (14.11%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 392/2772 레이블 처리 중 (14.14%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/

COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 516/2772 레이블 처리 중 (18.61%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 517/2772 레이블 처리 중 (18.65%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 518/2772 레이블 처리 중 (18.69%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 519/2772 레이블 처리 중 (18.72%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 520/2772 레이블 처리 중 (18.76%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 521/2772 레이블 처리 중 (18.80%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 522/2772 레이블 처리 중 (18.83%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 523/2772 레이블 처리 중 (18.87%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 524/2772 레이블 처리 중 (18.90%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/

...

COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2199/2772 레이블 처리 중 (79.33%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2200/2772 레이블 처리 중 (79.37%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2201/2772 레이블 처리 중 (79.40%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2202/2772 레이블 처리 중 (79.44%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2203/2772 레이블 처리 중 (79.47%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2204/2772 레이블 처리 중 (79.51%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2205/2772 레이블 처리 중 (79.55%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2206/2772 레이블 처리 중 (79.58%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2207/2772 레이블 처리 중 (79.62%), 에러: 0
COCOTOYOLO [/root/fashion_result_merg

COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2353/2772 레이블 처리 중 (84.88%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2354/2772 레이블 처리 중 (84.92%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2355/2772 레이블 처리 중 (84.96%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2356/2772 레이블 처리 중 (84.99%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2357/2772 레이블 처리 중 (85.03%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2358/2772 레이블 처리 중 (85.06%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2359/2772 레이블 처리 중 (85.10%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2360/2772 레이블 처리 중 (85.14%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2361/2772 레이블 처리 중 (85.17%), 에러: 0
COCOTOYOLO [/root/fashion_result_merg

COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2490/2772 레이블 처리 중 (89.83%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2491/2772 레이블 처리 중 (89.86%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2492/2772 레이블 처리 중 (89.90%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2493/2772 레이블 처리 중 (89.94%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2494/2772 레이블 처리 중 (89.97%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2495/2772 레이블 처리 중 (90.01%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2496/2772 레이블 처리 중 (90.04%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2497/2772 레이블 처리 중 (90.08%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2498/2772 레이블 처리 중 (90.12%), 에러: 0
COCOTOYOLO [/root/fashion_result_merg

COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2618/2772 레이블 처리 중 (94.44%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2619/2772 레이블 처리 중 (94.48%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2620/2772 레이블 처리 중 (94.52%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2621/2772 레이블 처리 중 (94.55%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2622/2772 레이블 처리 중 (94.59%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2623/2772 레이블 처리 중 (94.62%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2624/2772 레이블 처리 중 (94.66%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2625/2772 레이블 처리 중 (94.70%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2626/2772 레이블 처리 중 (94.73%), 에러: 0
COCOTOYOLO [/root/fashion_result_merg

COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2741/2772 레이블 처리 중 (98.88%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2742/2772 레이블 처리 중 (98.92%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2743/2772 레이블 처리 중 (98.95%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2744/2772 레이블 처리 중 (98.99%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2745/2772 레이블 처리 중 (99.03%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2746/2772 레이블 처리 중 (99.06%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2747/2772 레이블 처리 중 (99.10%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2748/2772 레이블 처리 중 (99.13%), 에러: 0
COCOTOYOLO [/root/fashion_result_merge->/root/fashion_converter] 진행 상황: 2749/2772 레이블 처리 중 (99.17%), 에러: 0
COCOTOYOLO [/root/fashion_result_merg

In [5]:
import os
import json
import shutil
from PIL import Image

unique_classes = set()

# 이미지 및 레이블이 있는 폴더 경로
data_folder = r'fashion_result_merge'

# YOLO 형식으로 변환한 데이터를 저장할 폴더 경로
output_folder = r'fashion_converter'

# 이미지 파일 확장자
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

# 현재 클래스들: baseball park, person, steeltower, towercrane, car, bike, truck, ground, helipad, etc vehicle, outdoor parking, bus, high-rise building
# YOLO 클래스 이름과 COCO 클래스 이름 매합 (필요에 따라 업데이트)
class_mapping = {
    "하의": {
        "청바지": 0,
        "팬츠": 1,
        "스커트": 2,
        "레깅스": 3,
        "조거팬츠": 4
        # 필요에 따라 다른 하의 종류를 추가로 매핑하십시오.
    },
    "아우터": {
        "코트": 5,
        "재킷": 6,
        "점퍼": 7,
        "패딩": 8,
        "베스트": 9,
        "가디건": 10,
        "짚업": 11
        # 필요에 따라 다른 아우터 종류를 추가로 매핑하십시오.
    },
    "원피스": {
        "드레스": 12,
        "점프수트": 13
        # 필요에 따라 다른 원피스 종류를 추가로 매핑하십시오.
    },
    "컬러": {
        "블랙": 14,
        "화이트": 15,
        "그레이": 16,
        "레드": 17,
        "핑크": 18,
        "오렌지": 19,
        "베이지": 20,
        "브라운": 21,
        "옐로우": 22,
        "그린": 23,
        "카키": 24,
        # 필요에 따라 다른 컬러를 추가로 매핑하십시오.
        "실버": 25
    },
    "스타일": {
        "스포티": 26,
        "톰보이": 27
    }
    # 필요에 따라 다른 카테고리를 추가로 매핑하십시오.
}

# 레이블을 YOLO 형식으로 변환
def coco_to_yolo_label(coco_label, points, width, height):
    yolo_class = class_mapping.get(coco_label[0], 'unknown')
    if len(points) == 4:
        x_min, y_min,x_max ,y_max  = points[0], points[1], points[2], points[3]
        x_center = (x_min + x_max) / (2 * width)
        y_center = (y_min + y_max) / (2 * height)
        box_width = (x_max - x_min) / width
        box_height = (y_max - y_min) / height
        return f"{yolo_class} {x_center} {y_center} {box_width} {box_height}"

# 이미지 파일과 레이블 파일의 수를 계산
total_image_files = 0
total_label_files = 0
error_files = []
# 진행 중인 파일 수 초기화
processed_files = 0

for root, _, files in os.walk(data_folder):
    for file in files:
        if file.lower().endswith(tuple(image_extensions)):
            total_image_files += 1
        elif file.lower().endswith('.json'):
            total_label_files += 1
        
for root, _, files in os.walk(data_folder):
    for file in files:
        if file.lower().endswith(tuple(image_extensions)):
            # 이미지 파일 경로
            image_path = os.path.join(root, file)

            # 해당 이미지에 대한 레이블 파일 경로
            label_name = os.path.splitext(file)[0]
            label_file = label_name + '.json'
            label_file_path = os.path.join(root, label_file)

            if os.path.isfile(label_file_path):
                yolo_labels = []  # 이미지에 대한 모든 레이블 정보를 저장하는 리스트
                with open(label_file_path, 'r', encoding='utf-8') as label_data:
                    coco_data = json.load(label_data)
                    # 이미지의 너비와 높이 가져오기
                    try:
                        width = coco_data["이미지 정보"]["이미지 너비"]
                        height = coco_data["이미지 정보"]["이미지 높이"]
                    except KeyError:
                        print(f"에러: 이미지 해상도를 찾을 수 없습니다 - {label_name}")
                        error_files.append(label_name)
                        continue

                    # 'annotation'이 리스트인 경우에 대한 처리
                    annotations_list = coco_data.get('데이터셋 정보', {}).get('데이터셋 상세설명', {}).get('렉트좌표', {})
                    if not isinstance(annotations_list, list):
                        annotations_list = [annotations_list]

                    for annotations in annotations_list:
                        #annotations_str = json.dumps(annotations)
                        print(annotations)
                        coco_label=coco_data.get('데이터셋 정보', {}).get('데이터셋 상세설명', {}).get('라벨링', {}).get(annotations,[])
                        if not isinstance(coco_label, list):
                            coco_label=[coco_label]
                        # 예외 처리: coco_label이 빈 리스트인 경우
                        if not coco_label:
                            continue  # 빈 리스트인 경우 스킵하고 다음 반복으로 이동
                        for annotation in annotations:
                            box_info = annotation
                            points = [
                                box_info.get('X좌표', 0),
                                box_info.get('Y좌표', 0),
                                box_info.get('가로', 0) + box_info.get('X좌표', 0),
                                box_info.get('세로', 0) + box_info.get('Y좌표', 0)]
                            print(annotation)


{'아우터': [{}], '하의': [{'X좌표': 329.722, 'Y좌표': 181.754, '가로': 104, '세로': 229}], '원피스': [{}], '상의': [{'X좌표': 341.278, 'Y좌표': 38.7923, '가로': 108, '세로': 237}]}


TypeError: unhashable type: 'dict'